In [1]:
import time

from env import Env
from dqn import DQN

In [2]:
# 这里我们定义多个游戏地图，后面可以使用不同的游戏地图观察agent的行为
def game_map_1(environment):
    environment.reset()
    environment.add_item('yellow_star', (3, 3), credit=100, pickable=True)
    environment.add_item('yellow_star', (0, 7), credit=100, pickable=True)
    environment.add_item('red_ball', (5, 6), terminal=True, label="Exit")

In [3]:
# set the environment
env = Env((8, 8), (130, 90), default_rewards=0)

In [4]:
# select a game
game_map_1(env)

In [5]:
for _ in range(1):
    action = env.action_space.sample()
    print(action)
    reward, next, end = env.step(action)
    print(reward, next, end)
    time.sleep(0.2)
    
env.reset()

N
0 (0, 0) False


In [6]:
# 下面两个函数将位置信息转换为状态信息
def location_one_hot(location, map_dimension):
    row, column = location
    total_rows, total_columns = map_dimension
    
    assert row < total_rows and column < total_columns
    
    # 将`行`和`列`合并为一个ID，后面用于`one hot`编码
    location_id = row * total_columns + column
    
    # `one hot`编码
    one_hot = [0] * (total_rows * total_columns)
    one_hot[location_id] = 1
    
    return one_hot

def location_multi_hot(locations, map_dimension):
    total_rows, total_columns = map_dimension
    one_hot = [0] * (total_rows * total_columns)
    
    for loc in locations:
        row, column = loc
        
        assert row < total_rows and column < total_columns
        
        # 将`行`和`列`合并为一个ID用于`one hot`编码
        location_id = row * total_columns + column
        one_hot[location_id] = 1
        
    return one_hot

# 下面函数将环境的全部信息转换成状态信息
def state_from_environment(environment):
    # 环境地图大小
    dimension = (environment.map.n_rows, environment.map.n_columns)

    # agent状态信息
    agent_state = location_one_hot(environment.agent.at, dimension)

    star_locations = []
    exit_location = None
    for item in environment.map.all_items:        
        if item.pickable == True:
            star_locations.append(item.index)
        elif item.terminal == True:
            exit_location = item.index
        else:
            assert False, "Unknown item in the environment"
            
    # 必须给agent设置一个出口
    assert exit_location != None, "You must have a exit point for agent!"

    # 出口Exit状态信息
    exit_state = location_one_hot(exit_location, dimension)

    # 环境中星星的状态信息
    stars_state = location_multi_hot(star_locations, dimension)

    # 返回所有信息的组合
    return agent_state + exit_state + stars_state

In [7]:
# hyperparameters
lr = 0.001
gamma = 0.99

training_episodes = 1

In [8]:
dqn = DQN(env.map.n_squares*3, env.action_space.n_actions, lr, gamma)

In [13]:
training_episodes = 5000
total_losses = 0
for episode in range(1, training_episodes+1):
    env.reset()  # 复位环境
    env.show = False
    
    # 此时环境刚复位，获取此时的环境状态信息
    state = state_from_environment(env)

    # 调式信息
    location = env.agent.at

    # 记录此回合agent的经历
    this_episode = []
    
    end = False  # 表明此回合是否结束
    while end == False:
        # 查询DQN由当前状态获取动作
        # 注意：使用DQN时一般将动作编码为从0开始的连续数字，DQN内部以及其输入输出
        # 都使用这种数字代表动作。
        # 环境理解的动作可能不是数字，所以要进行转换。
        action_id = dqn.next_action(state)
        action = env.action_space.action_from_id(action_id)

        # 指导agent走一步，环境返回这一步行动产生的reward，agent的位置和agent是否到达了出口
        reward, next_location, end = env.step(action)

        # 获取agent走了一步后的环境状态信息
        next_state = state_from_environment(env)

        # 记录这一步
        this_episode.append((state, action_id, reward, next_state))

        state = next_state

        # 调式信息
        #print("action {}: {} ----> {} reward {}".format(action, location, next_location, reward))
        #print(next_state)
        location = next_location

    # 训练DQN
    loss = dqn.train_an_episode(this_episode)
    total_losses += loss
    if episode != 0 and episode % 200 == 0:
        print("trained episodes {}: current loss is {:.4f}, average loss is {:.4f}".format(episode, loss, total_losses/episode))

trained episodes 200: current loss is 5.5052, average loss is 21.8122
trained episodes 400: current loss is 11.5739, average loss is 19.1100
trained episodes 600: current loss is 4.7787, average loss is 18.9909
trained episodes 800: current loss is 1.6400, average loss is 17.9131
trained episodes 1000: current loss is 5.8743, average loss is 17.6030
trained episodes 1200: current loss is 3.5462, average loss is 16.9091
trained episodes 1400: current loss is 11.3917, average loss is 16.5325
trained episodes 1600: current loss is 1.2886, average loss is 15.8282
trained episodes 1800: current loss is 12.9774, average loss is 15.6585
trained episodes 2000: current loss is 3.0578, average loss is 15.1230
trained episodes 2200: current loss is 2.3314, average loss is 14.7498
trained episodes 2400: current loss is 3.2766, average loss is 14.5229
trained episodes 2600: current loss is 5.5554, average loss is 14.1429
trained episodes 2800: current loss is 2.7633, average loss is 13.8948
trained